# Data retrieval from localbitcoins API

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone

In [2]:
def make_request(url_base,url):
    r = requests.get(url_base+url)
    file_json = r.json()
    return file_json

def json_to_pandas(file_json:dict):
    return pd.DataFrame.from_dict(file_json)


### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-10"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

## Currencies

In [3]:
url_base = 'https://localbitcoins.com/'
# Currencies:
url = "api/currencies"
currency_code = make_request(url_base,url)
df_currency_code = json_to_pandas(currency_code['data'])
# df_currency_code.info()
# df_currency_code.head()

In [4]:
## Extract country information related to currency
currency_name = []
currency_is_altcoin = []
for index, row in df_currency_code.iterrows():
    currency_name.append(row[0]["name"])
    currency_is_altcoin.append(row[0]["altcoin"])
    
## Reorganize on df

df_currency_code['currencies'] = currency_name
df_currency_code['altcoin'] = currency_is_altcoin
df_currency_code = df_currency_code.reset_index()

df_currency_code.head()

,index,currencies,currency_count,altcoin
0,ADA,Cardano (ADA),185,True
1,AED,United Arab Emirates Dirham (AED),185,False
2,AFN,Afghan Afghani (AFN),185,False
3,ALL,Albanian Lek (ALL),185,False
4,AMD,Armenian Dram (AMD),185,False


### Average price of btc by country (in local currency)

In [5]:
url = "bitcoinaverage/ticker-all-currencies/"
price_btc = make_request(url_base,url)
time_stamp = datetime.now().timestamp()

In [6]:
df_price_btc = json_to_pandas(price_btc)

#transpose data frame. Index is now by Country:

df_price_btc = df_price_btc.T
df_price_btc['time_stamp'] = time_stamp
df_price_btc = df_price_btc.reset_index()
print(df_price_btc.info())
df_price_btc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       72 non-null     object 
 1   volume_btc  72 non-null     object 
 2   rates       72 non-null     object 
 3   avg_1h      35 non-null     object 
 4   avg_6h      53 non-null     object 
 5   avg_12h     58 non-null     object 
 6   avg_24h     72 non-null     object 
 7   time_stamp  72 non-null     float64
dtypes: float64(1), object(7)
memory usage: 4.6+ KB
None


,index,volume_btc,rates,avg_1h,avg_6h,avg_12h,avg_24h,time_stamp
0,EUR,4.28949073,{'last': '38709.52'},34016.07,34522.10,34572.21,34585.95,1.643287e+09
1,ZAR,2.00751860,{'last': '566443.87'},570024.94,568705.98,567729.49,575202.97,1.643287e+09
2,RUB,4.25682717,{'last': '3220022.69'},2985101.11,2976343.38,2976776.34,3026467.32,1.643287e+09
3,NGN,2.83831194,{'last': '21959802.58'},21083975.43,20897019.62,20905762.91,21298933.54,1.643287e+09
4,COP,3.95777454,{'last': '150000000.00'},149551898.77,148218983.17,147704780.00,151326424.59,1.643287e+09


## Merge and clean dataframe

In [7]:
df_market = df_price_btc.merge(df_currency_code, how='right')

# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

# Add date from time_stamp
df_market['date'] = datetime.fromtimestamp(time_stamp, tz=timezone.utc)
df_market = df_market[['time_stamp','date','index','currencies','volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','altcoin']]
df_market.rename(columns = {'index':'currency_code'}, inplace = True)
print(df_market.info())
df_market.head(10)


### I should include some data formatting so info is stored in the right format

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 1 to 183
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     72 non-null     float64            
 1   date           72 non-null     datetime64[ns, UTC]
 2   currency_code  72 non-null     object             
 3   currencies     72 non-null     object             
 4   volume_btc     72 non-null     object             
 5   avg_24h        72 non-null     object             
 6   avg_12h        58 non-null     object             
 7   avg_6h         53 non-null     object             
 8   avg_1h         35 non-null     object             
 9   altcoin        72 non-null     bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), object(7)
memory usage: 5.7+ KB
None


,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin
1,1.643287e+09,2022-01-27 12:36:26.977529+00:00,AED,United Arab Emirates Dirham (AED),0.00783186,140451.95,NaN,NaN,NaN,False
6,1.643287e+09,2022-01-27 12:36:26.977529+00:00,AOA,Angolan Kwanza (AOA),0.00691542,26028787.84,NaN,NaN,NaN,False
7,1.643287e+09,2022-01-27 12:36:26.977529+00:00,ARS,Argentine Peso (ARS),0.61736599,8359807.93,8122377.39,8234065.88,7802860.72,False
8,1.643287e+09,2022-01-27 12:36:26.977529+00:00,AUD,Australian Dollar (AUD),0.67919046,52361.57,51552.30,53541.06,55011.65,False
15,1.643287e+09,2022-01-27 12:36:26.977529+00:00,BDT,Bangladeshi Taka (BDT),0.00525984,2999992.39,2999992.39,3007472.14,NaN,False
21,1.643287e+09,2022-01-27 12:36:26.977529+00:00,BOB,Bolivian Boliviano (BOB),0.09305072,275282.88,271632.86,NaN,NaN,False
22,1.643287e+09,2022-01-27 12:36:26.977529+00:00,BRL,Brazilian Real (BRL),0.82718619,206713.11,205648.97,205818.61,205816.21,False
26,1.643287e+09,2022-01-27 12:36:26.977529+00:00,BYN,Belarusian Ruble (BYN),0.39576458,101918.19,101281.71,101466.31,99954.46,False
28,1.643287e+09,2022-01-27 12:36:26.977529+00:00,CAD,Canadian Dollar (CAD),1.27921728,48453.03,50069.56,50812.37,51860.78,False
29,1.643287e+09,2022-01-27 12:36:26.977529+00:00,CDF,Congolese Franc (CDF),0.00041943,95367522.59,NaN,NaN,NaN,False


In [8]:
## Adding avg_24h exchange rate USD/BTC for inmediate comparison:

df_market["avg_24h_usd"] = df_market[df_market['currency_code']=="USD"].avg_24h.tolist()[0]

In [9]:
### Formatting data types:

# Float values:
for column_label in ['volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','avg_24h_usd']:

    df_market[column_label] = pd.to_numeric(df_market[column_label])

df_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 1 to 183
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     72 non-null     float64            
 1   date           72 non-null     datetime64[ns, UTC]
 2   currency_code  72 non-null     object             
 3   currencies     72 non-null     object             
 4   volume_btc     72 non-null     float64            
 5   avg_24h        72 non-null     float64            
 6   avg_12h        58 non-null     float64            
 7   avg_6h         53 non-null     float64            
 8   avg_1h         35 non-null     float64            
 9   altcoin        72 non-null     bool               
 10  avg_24h_usd    72 non-null     float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(7), object(2)
memory usage: 6.3+ KB


In [10]:
## Calculating implicit exchange rate:
df_market['implicit_exchange'] = df_market['avg_24h'].divide(df_market["avg_24h_usd"], fill_value = None)

In [11]:
# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

In [12]:
df_market.head(2)

,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin,avg_24h_usd,implicit_exchange
1,1.643287e+09,2022-01-27 12:36:26.977529+00:00,AED,United Arab Emirates Dirham (AED),0.007832,140451.95,NaN,NaN,NaN,False,39380.01,3.56658
6,1.643287e+09,2022-01-27 12:36:26.977529+00:00,AOA,Angolan Kwanza (AOA),0.006915,26028787.84,NaN,NaN,NaN,False,39380.01,660.96448


## Save information in .csv and sql database

In [13]:
# Saving csv by timestamp:
df_market.to_csv(f'./data/csv/{time_stamp}.csv',index=False)
# Saving historical data by appending:
df_market.to_csv(f'./data/csv/historical_data.csv',index=False, mode = 'a')

In [14]:
# Append information on sqlite database

from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///data/CurrenciesDataBase.db')
print(type(engine))
df_market.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


#### Calculation of pct_change taking 2021-10-03 as date of reference

In [15]:
# path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
# df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

# ### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# # The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# # are done in the full dataset everytime the code runs.
# # This practice is not scalable

# # The pct could be calculated on the web interface for a given date selected by the users but this implies 
# # more processing time

In [16]:
# df = df_market.groupby(by=["date","currency_code"]).mean()
# df.reset_index(inplace=True)

# ### Dataset with implicit exchange of reference by currency:
# date_reference = "2021-10-03"
# exchange = exchange_reference(df, date_reference)
# exchange = pd.DataFrame.from_dict(exchange, orient="index")

# df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

# ### Drop Null values:

# df.dropna(subset=['pct'], inplace=True)

In [17]:
# df.info()

In [18]:
# ### Saving on DB Version 2

# engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
# print(type(engine))
# df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

In [19]:
now = dt.datetime.now()
print("Last Execution: ")
print(str(now))
print('Completed')

Last Execution: 
2022-01-27 13:36:27.370377
Completed
